In [ ]:
import pandas as pd
import re
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Global Variables
Lib_Name = 'Lib_1' #UPDATE with library name or base name you want for your files
Fig_Format = 'jpeg' #UPDATE default graph file format

fastq_file_path = 'path/to/paired/fastq/file'#UPDATE with file path to your paired seq reads (fastq)

design_file_txt = 'path/to/design_file.txt'#UPDATE with design file containing your designed tiles as a txt file

Name_Rep1 = 'Rep1' #UPDATE default is rep1 but I like to change to sequencing index number 
Name_Rep2 = 'Rep2'#UPDATE default is rep2 but I like to change to sequencing index number 

Output_Directory = f'{Lib_Name}_Maps_and_Graphs'
os.makedirs(Output_Directory, exist_ok=True)

#create summary tabble
summary_dict = {'Category': [], 'Read Count': []}

In [ ]:
def find_designed(des):
    """Creates a lookup dictionary of all designed tiles from a file."""
    dt = []
    with open(des, 'r') as f_des:
        for line in f_des:
            leftReplace = line.replace("CCCAGCTTAAGCCACCATG", "") #UPDATE to match seq flanking your tiles in the design file, usually primer homology extensions
            rightReplace = leftReplace.replace("gGATCCGAGCTCGCTAGC\n", "") #UPDATE to match seq flanking your tiles in the design file, usually primer homology extensions
            dt.append(rightReplace.strip())
    return {tile: 1 for tile in dt}

def getmid(seq, pre, post):
    """Extracts the sequence between pre and post substrings."""
    match = re.search(f"{pre}(.*){post}", seq)
    return match.group(1) if match else "X" #puts and X if the pre and/or post seq cannot be found 
    
def tilebc_mapper(readfile, dtd, t_len=120, bc1_len=11, tile_pre="CACCATG", tile_post="GGATCCG",
                  bc1_pre="CGCTAGC", bc1_post="CTCGAGA"):  #UPDATE you need to change the correct tile length (t_len)and BC1 length (bc1_len) and the pre and post sequences flanking them
    """Processes input sequences to map tiles and barcodes."""  
    
    # Lists to store extracted data  
    tile_list, tile_lengths, tq_list, des_query = [], [], [], []  
    bc1_list, bc1_lengths, bc1q_list = [], [], []  
    total_sequences = 0  # Track the number of reads processed  
    sequences =[]

    with open(readfile, 'r') as fin:  # Reads paired fastq file and extracts the reads  
        for line in fin:  
            if line.startswith('@'):  # Identifies the sequence header  
                seq = next(fin).strip()  # Reads the actual sequence  
                sequences.append(seq) # adds seq to reads list 
                total_sequences += 1  

                # Identify Tile and BC1 based on pre and post sequences and check length  
                tile = getmid(seq, tile_pre, tile_post)  
                tile_len = len(tile)  
                tile_quality = 1 if tile_len == t_len else 0  # Quality column given 1 if length matches expected length, otherwise 0  
                tile_is_designed = 1 if tile in dtd else 0  # Checks if the tile is in the design dictionary  

                adBC = getmid(seq, bc1_pre, bc1_post)  
                adBC_len = len(adBC)  
                adBC_quality = 1 if adBC_len == bc1_len else 0  # Quality column for BC1, 1 if length matches expected length otherwise 0  

                # Store extracted values  
                tile_list.append(tile)  
                tile_lengths.append(tile_len)  
                tq_list.append(tile_quality)  
                des_query.append(tile_is_designed)  

                bc1_list.append(adBC)  
                bc1_lengths.append(adBC_len)  
                bc1q_list.append(adBC_quality)  

    # Create DataFrame containing all extracted information  
    tileBC_df = pd.DataFrame({  
        "Reads": sequences,
        "Tiles": tile_list,  
        "T Len": tile_lengths,  
        "T Qual": tq_list,  
        "Designed": des_query,  
        "AD BCs": bc1_list,  
        "A Len": bc1_lengths,  
        "A Qual": bc1q_list  
    })   

    return tileBC_df

In [ ]:
def process_maps(input_file, design_file):
    designed_tile_dict = find_designed(design_file)
    map1 = tilebc_mapper(input_file, designed_tile_dict)

In [ ]:
process_maps(fastq_file_path, design_file_txt)

### Output df has all reads from the seqfile

map1 is the original df

In [ ]:
#export the Map1 LUT
map1.to_csv(os.path.join(Output_Directory, f'{Lib_Name}_map1.csv'), index=False)

In [ ]:
#below are seires of analysis on the loook up table that was created 

In [ ]:
#tiles in design file
summary_dict['Category'].append('Tiles in Design File')
summary_dict['Read Count'].append(len(designed_tile_dict))
print(f'Number of Tiles in Design file {len(designed_tile_dict)}')

In [ ]:
#count reads with correct tile length, BC lenngth, and are in designed file
count_rows_t = ((map1['T Qual'] == 1)).sum()

summary_dict['Category'].append(f'Reads with Correct Tile Length')
summary_dict['Read Count'].append(count_rows_t)

print("Number of rows with 1 in T Qual :", count_rows_t)

In [ ]:
#count reads with correct tile length, BC lenngth, and are in designed file
count_rows_d = ((map1['Designed'] == 1)).sum()

summary_dict['Category'].append(f'Reads with Tile in the design file')
summary_dict['Read Count'].append(count_rows_d)

print("Number of rows with 1 in Designed :", count_rows_d)

In [ ]:
#count reads with correct tile length, BC lenngth, and are in designed file
count_rows_a = ((map1['A Qual'] == 1)).sum()

summary_dict['Category'].append(f'Reads with Correct BC1 Length')
summary_dict['Read Count'].append(count_rows_a)

print("Number of rows with 1 in A Qual :", count_rows_a)

In [ ]:
#count reads with correct tile length, BC lenngth, and are in designed file
count_rows_one = len(map1[(map1['T Qual'] == 1) & (map1['A Qual'] == 1)])

summary_dict['Category'].append(f'Rows with correct Tile length and BC1 length')
summary_dict['Read Count'].append(count_rows_one)

print("Number of rows with 1 in T Qual and A Qual:", count_rows_one)

In [ ]:
#count reads with correct tile length, BC lenngth, and are in designed file
count_rows = len(map1[(map1['T Qual'] == 1) & (map1['A Qual'] == 1) & (map1['Designed'] == 1)])

summary_dict['Category'].append(f'Rows with correct Tile length, BC1 length, and Tile is in design file')
summary_dict['Read Count'].append(count_rows)

print("Number of rows with 1 in T Qual, A Qual, and Designed:", count_rows)

In [ ]:
#plot tile length histogram
plt.hist(map1['T Len'])
plt.xlim([0, 200]) #UPDATE if your tile length won't fit in this range
plt.title(f'{Lib_Name } Tile Length Frequency')
plt.xlabel('T Length')
plt.ylabel('Frequency')
plt.savefig(os.path.join(Output_Directory, f'{Lib_Name}_T_Length.{Fig_Format}'))
plt.show()


In [ ]:
#plot tile qual as histogram
plt.hist(map1['T Qual'])
plt.title(f'{Lib_Name } Tile Quality Frequency')
plt.xlabel('T Qual')
plt.ylabel('Frequency')
plt.savefig(os.path.join(Output_Directory, f'{Lib_Name}_T_Quality.{Fig_Format}'))
plt.show()

In [ ]:
# plot AD BC len
plt.hist(map1['A Len'], bins=100)
plt.xlim([0, 20]) #UPDATE if your bc1 length won't fit in this range
plt.title(f'{Lib_Name } BC1 Length Frequency')
plt.xlabel('BC1 Length')
plt.ylabel('Frequency')
plt.savefig(os.path.join(Output_Directory, f'{Lib_Name}_bc1_length.{Fig_Format}'))
plt.show()

In [ ]:
#plot bc1 qual as histogram

plt.hist(map1['A Qual'])
plt.title(f'{Lib_Name } BC1 Quality Frequency')
plt.xlabel('A Qual')
plt.ylabel('Frequency')
plt.savefig(os.path.join(Output_Directory, f'{Lib_Name}_bc1_Quality.{Fig_Format}'))
plt.show()

### Filtering out reads that either do not have the consensus sequences to find the tiles/BCs OR have tiles/BCs of unexpected lengths

map2 is the df with only tiles that match designed and where BC lengths are as expected. All length and quality columns are dropped, as well as the column that stored the full read strings.

In [ ]:

#Replace all 0s in map1 with NaN to filter out any Qual=0 reads
map1_nans = map1.replace(0, np.nan)
map2 = map1_nans.dropna().reset_index()

#get rid of some now useless columns
clabels = ['index','Reads', 'T Len','T Qual', 'Designed', 'A Len','A Qual']
map2 = map2.drop(clabels, axis = 1)


# how many reads are lost? rc = readcount
rcmap1 = map1.shape[0]
rcmap2 = map2.shape[0]
diffpct = ((rcmap1 - rcmap2) / rcmap1)*100
print("% Reads lost:")
print(diffpct)

summary_dict['Category'].append(f'Map1 Shape')
summary_dict['Read Count'].append(rcmap2)


### Coverage
map3 is a df with tiles, BCs, and combos of tiles and BCs

In [ ]:
map3 = map2.copy()
map3.head()

In [ ]:
#Add column that connects BC1 to the Tile it is paired with (Cat for concatenation) 
adcol = map3['AD BCs'].copy()

map3['Cat'] = map3['Tiles'].str.cat(adcol, sep="-")

summary_dict['Category'].append(f'Map3 Shape')
summary_dict['Read Count'].append(map3.shape[0])

In [ ]:
#Frequency of each tile-bc combo

tbcov = map3['Cat'].value_counts().to_frame().reset_index()

summary_dict['Category'].append(f'Unique Tile+BC1 coverage')
summary_dict['Read Count'].append(tbcov.shape[0])

print(f'number unique tb combos:{tbcov.shape[0]}')



In [ ]:
# plot tb coverage on histogram
plt.figure()
plt.title(f'{Lib_Name} Unique Tile + BC Read Coverage Frequency')
plt.hist(tbcov['Cat'],  bins=75)
plt.xlabel('Coverage')
plt.ylabel('Counts')
plt.savefig(os.path.join(Output_Directory, f'{Lib_Name}}_tbc1_cov.{Fig_Format}'))
plt.show()

In [ ]:
# Unique Tile coverage
tcov = map2['Tiles'].value_counts().to_frame().reset_index()

summary_dict['Category'].append(f'Unique Tile coverage')
summary_dict['Read Count'].append(tcov.shape[0])

print (f'Unique Tiles: {tcov.shape[0]}')

In [ ]:
# plot tile coverage 
plt.hist(tcov['Tiles'], bins=100)
plt.title('Unique Tile Coverage Frequency')
plt.xlabel('Coverage')
plt.ylabel('Counts')
plt.savefig(os.path.join(Output_Directory, f'{Lib_Name}}_t_cov.{Fig_Format}'))
plt.show()

In [ ]:
#Unique ad bc coverage
abcov = map3['AD BCs'].value_counts().to_frame().reset_index()

summary_dict['Category'].append(f'Unique bc1')
summary_dict['Read Count'].append(abcov.shape[0])

print(f'Unique BC1: {abcov.shape[0]}') # number unique ad bcs


In [ ]:
# plot ad bc coverage 
plt.hist(abcov['AD BCs'], bins=100)
plt.title(f'{Lib_Name} Unique BC1 Read Coverage Frequency')
plt.xlabel('Coverage')
plt.ylabel('Counts')
plt.savefig(os.path.join(Output_Directory, f'{Lib_Name}}_bc1_cov.{Fig_Format}'))
plt.show()

In [ ]:
#make csv of map3
map3.to_csv(os.path.join(Output_Directory, f'{Lib_Name}_map3.csv'), index=False)

In [ ]:
#create summary table
summary_dict_df = pd.DataFrame.from_dict(summary_dict)
summary_dict_df.to_csv(os.path.join(Output_Directory, f'{Lib_Name}_LUT_Summary.csv'), index=False)